In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error




In [2]:
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet > data/yellow_tripdata_2023-01.parquet
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet > data/yellow_tripdata_2023-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.4M  100 45.4M    0     0  21.7M      0  0:00:02  0:00:02 --:--:-- 21.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.5M  100 45.5M    0     0  21.7M      0  0:00:02  0:00:02 --:--:-- 21.7M


In [3]:
df_jan = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

### Q1. Downloading the data

```
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

16
17
18
19




In [4]:
f"columns for jan data are {df_jan.shape[1]}"

'columns for jan data are 19'

### Q2. Computing duration
```
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

32.59
42.59
52.59
62.59
```

In [5]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [6]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

df_feb['duration'] = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)



In [7]:
f"std for jan data is {df_jan['duration'].std()}" 


'std for jan data is 42.59435124195458'

# Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

- 90%
- 92%
- 95%
- 98%

In [8]:
# Calculate fraction of remaining records
original_jan_count = len(df_jan)
original_feb_count = len(df_feb)

filtered_jan_count = len(df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)])
filtered_feb_count = len(df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)])

jan_fraction = filtered_jan_count / original_jan_count
feb_fraction = filtered_feb_count / original_feb_count

print(f"January fraction of records remaining: {jan_fraction:.3f}")
print(f"February fraction of records remaining: {feb_fraction:.3f}")

January fraction of records remaining: 0.981
February fraction of records remaining: 0.980


### Q4. One-hot encoding

```
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

2
155
345
515
715
```

In [9]:
categorical = ['PULocationID', 'DOLocationID']
#numerical = ['trip_distance']


df_jan[categorical] = df_jan[categorical].astype(str)
print(df_jan.shape)
print(df_jan.dtypes)

(3066766, 20)
VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                     object
DOLocationID                     object
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object


In [10]:
#train_dicts = df_jan[categorical + numerical].to_dict(orient='records')
train_dicts = df_jan[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

print(f"Number of columns in feature matrix: {X_train.shape[1]}")




Number of columns in feature matrix: 518


### Q5. Training a model

```
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

3.64
7.64
11.64
16.64
```

In [11]:
#train_dicts = df_jan[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

print(f"Number of columns in feature matrix: {X_train.shape[1]}")


target = 'duration'
y_train= df_jan[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mse = mean_squared_error(y_train, y_pred)


#rmse = np.sqrt(mse)
rmse = root_mean_squared_error(y_train, y_pred)


print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")

Number of columns in feature matrix: 518
Mean Squared Error: 1763.705819778005
Root Mean Squared Error: 41.9964977084757


### Q6. Evaluating the model

```
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

3.81
7.81
11.81
16.81
```


In [12]:

#val_dicts = df_feb[categorical + numerical].to_dict(orient='records')
val_dicts = df_feb[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
print(df_feb.shape)

print(f"Number of columns in feature matrix: {X_val.shape[1]}")

target = 'duration'
y_val = df_feb[target].values


y_pred = lr.predict(X_val)

mse = mean_squared_error(y_val, y_pred)


rmse = np.sqrt(mse)
#rmse = root_mean_squared_error(X_val, y_pred)



print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")

(2913955, 20)
Number of columns in feature matrix: 518
Mean Squared Error: 1935.0926276563732
Root Mean Squared Error: 43.989687742201276
